**Updating counts with new transit stations**

In [10]:
import pandas as pd
from geopy.distance import distance

# Load final dataset containing counts and junctions duplicated for every year
final_dataset = pd.read_excel('final_dataset_count.xlsx')

# Filter junctions for the year 2023
junctions_2023 = final_dataset[final_dataset['year'] == 2023]

# Load new transit stations dataset
transit_stations = pd.read_excel('transit_stations.xlsx')

# Filter for the new stations
new_stations = transit_stations[transit_stations['year'] == 2027]

# Function to calculate distance using geopy.distance
def calculate_nearest_junction(station_row, junctions_df):
    station_coords = (station_row['latitude'], station_row['longitude'])
    distances = junctions_df.apply(lambda row: distance(station_coords, (row['latitude'], row['longitude'])).km, axis=1)
    closest_junction_index = distances.idxmin()
    return junctions_df.loc[closest_junction_index, 'id']

# Find the nearest junction for each transit station
new_stations['Closest_Junction_ID'] = new_stations.apply(calculate_nearest_junction, axis=1, junctions_df=junctions_2023)

# Calculate updated transit counts based on closest junctions
transit_counts_by_junction = new_stations['Closest_Junction_ID'].value_counts().reset_index()
transit_counts_by_junction.columns = ['id', 'Updated_Transit_Count']

# Merge with junctions_2023 to get all junctions with their updated transit counts
junctions_with_transit_counts = junctions_2023.merge(transit_counts_by_junction, on='id', how='left')
junctions_with_transit_counts['Updated_Transit_Count'].fillna(0, inplace=True)
junctions_with_transit_counts['Updated_Transit_Count'] = junctions_with_transit_counts['Updated_Transit_Count'].astype(int)

# Optionally, you may want to combine previous transit counts with the updated counts
# Assuming 'transit_Count' is the column in junctions_2023 containing original counts
junctions_with_transit_counts['Combined_Transit_Count'] = junctions_with_transit_counts['transit_count'] + junctions_with_transit_counts['Updated_Transit_Count']

# Delete 'transit_count' and 'Updated_Transit_Count' columns
junctions_with_transit_counts.drop(columns=['transit_count', 'Updated_Transit_Count'], inplace=True)

# Rename 'Combined_Transit_Count' to 'transit_count'
junctions_with_transit_counts.rename(columns={'Combined_Transit_Count': 'transit_count'}, inplace=True)

# Change all values in year column to 2024
junctions_with_transit_counts.loc[:, 'year'] = 2024

# Save the updated dataset with junctions and their transit counts to a new file
junctions_with_transit_counts.to_excel('future_dataset_transit_counts.xlsx', index=False)

print("The updated dataset with transit counts based on nearest junctions has been saved as 'future_dataset_transit_counts.xlsx'.")


C:\Users\domin\AppData\Local\Temp\ipykernel_18848\186224853.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_stations['Closest_Junction_ID'] = new_stations.apply(calculate_nearest_junction, axis=1, junctions_df=junctions_2023)
C:\Users\domin\AppData\Local\Temp\ipykernel_18848\186224853.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, t

The updated dataset with transit counts based on nearest junctions has been saved as 'future_dataset_transit_counts.xlsx'.


**Store Counts**

*Cleaning the business-licences*

In [14]:
import pandas as pd

# Load business licences data
business_licences = pd.read_excel('business-licences.xlsx')

# Load cleaned and transformed business licences data
cleaned_transformed_business_licences = pd.read_excel('cleaned_transformed_business_licences.xlsx')

# Step 1: Filter out business types not present in cleaned_transformed_business_licences
valid_business_types = cleaned_transformed_business_licences['businesstype'].unique()
filtered_business_licences = business_licences[business_licences['BusinessType'].isin(valid_business_types)]

# Step 2: Filter by city 'Vancouver'
filtered_business_licences = filtered_business_licences[filtered_business_licences['City'] == 'Vancouver']

# Step 3: Select rows that do not have an empty value for 'geo_point_2d'
filtered_business_licences = filtered_business_licences.dropna(subset=['geo_point_2d'])

# Step 4: Drop rows where 'IssuedDate' or 'ExpiredDate' is NaN
filtered_business_licences = filtered_business_licences.dropna(subset=['IssuedDate', 'ExpiredDate'])

# Step 5: Extract rows where 'IssuedDate' starts with '2024'
filtered_business_licences = filtered_business_licences[filtered_business_licences['IssuedDate'].str.startswith('2024')]

# Step 6: Remove rows where 'ExpiredDate' is not '2024-12-31'
filtered_business_licences = filtered_business_licences[filtered_business_licences['ExpiredDate'] == '2024-12-31']

# Save the cleaned data to a new Excel file
filtered_business_licences.to_excel('cleaned_business_2024.xlsx', index=False)

print("The cleaned business licences data has been saved as 'cleaned_business_2024.xlsx'.")


The cleaned business licences data has been saved as 'cleaned_business_2024.xlsx'.


*Updating store counts*

In [16]:
import pandas as pd
from geopy.distance import distance

# Load future dataset containing junctions and their transit counts
junctions_with_transit_counts = pd.read_excel('future_dataset_transit_counts.xlsx')

# Load cleaned business licenses data
cleaned_business_licenses = pd.read_excel('cleaned_business_2024.xlsx')

# Extract latitude and longitude from 'geo_point_2d'
cleaned_business_licenses[['latitude', 'longitude']] = cleaned_business_licenses['geo_point_2d'].str.split(',', expand=True).astype(float)

# Function to calculate the nearest junction for a given store
def calculate_nearest_junction(store_row, junctions_df):
    store_coords = (store_row['latitude'], store_row['longitude'])
    distances = junctions_df.apply(lambda row: distance(store_coords, (row['latitude'], row['longitude'])).km, axis=1)
    closest_junction_index = distances.idxmin()
    return junctions_df.loc[closest_junction_index, 'id']

# Find the nearest junction for each store
cleaned_business_licenses['Closest_Junction_ID'] = cleaned_business_licenses.apply(calculate_nearest_junction, axis=1, junctions_df=junctions_with_transit_counts)

# Calculate updated store counts based on closest junctions
store_counts_by_junction = cleaned_business_licenses['Closest_Junction_ID'].value_counts().reset_index()
store_counts_by_junction.columns = ['id', 'Updated_Store_Count']

# Merge with junctions_with_transit_counts to get all junctions with their updated store counts
junctions_with_store_counts = junctions_with_transit_counts.merge(store_counts_by_junction, on='id', how='left')
junctions_with_store_counts['Updated_Store_Count'].fillna(0, inplace=True)
junctions_with_store_counts['Updated_Store_Count'] = junctions_with_store_counts['Updated_Store_Count'].astype(int)

# Combine previous store counts with the updated counts
# Assuming 'store_count' is the column in junctions_with_transit_counts containing original counts
junctions_with_store_counts['Combined_Store_Count'] = junctions_with_store_counts['store_count'] + junctions_with_store_counts['Updated_Store_Count']

# Delete 'store_count' and 'Updated_Store_Count' columns
junctions_with_store_counts.drop(columns=['store_count', 'Updated_Store_Count'], inplace=True)

# Rename 'Combined_Store_Count' to 'store_count'
junctions_with_store_counts.rename(columns={'Combined_Store_Count': 'store_count'}, inplace=True)

# Save the updated dataset with junctions and their store counts to a new file
junctions_with_store_counts.to_excel('future_dataset_count.xlsx', index=False)

print("The updated dataset with store counts based on nearest junctions has been saved as 'future_dataset_count.xlsx'.")


C:\Users\domin\AppData\Local\Temp\ipykernel_18848\1727660218.py:29: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  junctions_with_store_counts['Updated_Store_Count'].fillna(0, inplace=True)


The updated dataset with store counts based on nearest junctions has been saved as 'future_dataset_count.xlsx'.


**Calculating Reaches**

In [3]:
# Calculate the reaches of the time invariant features

import sys
sys.path.append('../') # This should probably be changed to a more sofisticated system at some point. i.e. install the package

import math

from heapq import heappush, heappop

from ast import literal_eval
from data_wrangler.dataset import Dataset

JUNCTION_FILE = 'future_dataset_count.csv'

CRIME_SIGMA = 132
STANDARD_DEVIATION = 400

junctions = Dataset.load_file(JUNCTION_FILE)
junctions.convert_properties({
    'id': int,
    'crime_count': int,
    'store_count': int,
    'police_count': int,
    'transit_count': int,
    'graffiti_count': int,
    'homeless_shelter_count': int,
    'traffic_signal_count': int,
    'street_lighting_poles_count': int,
    'schools_count': int,
    'neighbors': lambda v : literal_eval(v) if v else []
})

def normal_dst(distance, standard_deviation):
    scale = 1 / (2 * math.pi * (standard_deviation ** 2))
    power = distance ** 2 / (2 * standard_deviation ** 2)
    distribution = math.exp(-power)
    return scale * distribution

def reach_dst(distance, scale):
    """ Calculate a modified version of Borgatti's reach formula
    
    TODO: Check that convergence is important and that if is whether we actually need to cube the denominator
    The range formula is:  sumweight * 1 / (dst_scale * dst + 1) ^ 2
    We have +1 because we want distance of zero to be constant with respect to dst_scale
    We cube the denominator because this causes it to converge
    """
    
    return 1 / ((distance / scale + 1) ** 3)

def calculate_reach(junction, properties, dst_func, limit=float('inf')):
    """
    Args:
        junction (Row): The junction to calculate the reach for
        prop (str): The property to use for junction weights
        dst_scale (float): The value to scale distance by. Should be in the range (0, 1]. Likely close to zero.

    Returns:
        float: The calculated reach.
    """
    reaches = { key: 0 for key in properties}
    visited = set()
    queue = []
    heappush(queue, (0, junction['id']))
    while queue:
        dst, next_jun = heappop(queue)
        if next_jun in visited: continue
        visited.add(next_jun)
        if dst > limit: continue
        
        # The range formula is: weight * 1 / (dst_scale * dst + 1) ^ 2
        # We have +1 because we want distance of zero to be constant with respect to dst_scale
        # We square the denominator because this causes it to converge
        
        # Update the range values
        crime_dst = normal_dst(dst, CRIME_SIGMA)
        scaled_dst = dst_func(dst)
        for key in properties:
            if key == 'crime_reach':
                reaches[key] += junctions[next_jun][properties[key]] * crime_dst
            else:
                reaches[key] += junctions[next_jun][properties[key]] * scaled_dst
              
        for neighbor, delta, s_id in junctions[next_jun]['neighbors']:
            if neighbor in visited: continue
            neighbor_dst = dst + delta
            heappush(queue, (neighbor_dst, neighbor))
    return reaches

def calculate_reaches(junctions, properties, dst_func, limit=float('inf')):
    highest = { key: 0 for key in properties}
    
    for i, junction in enumerate(junctions):
        reaches = calculate_reach(junction, properties, dst_func, limit)
        for key in reaches:
            junction[key] = reaches[key]
            highest[key] = max(highest[key], reaches[key])
        
        if (i+1) % 100 == 0:
            print(f'\rCalculated {i+1}/{len(junctions)}           ', end='')
    print(f'\rCalculated {len(junctions)}/{len(junctions)}        ')
    print("Normalizing")
    for junction in junctions:
        for key in properties:
            junction[key] /= highest[key]
    print("Done")
    
calculate_reaches(
    junctions, 
    {
        # 'crime_reach': 'crime_count',
        'store_reach': 'store_count',
        # 'police_reach': 'police_count',
        'transit_reach': 'transit_count',
        # 'graffiti_reach': 'graffiti_count',
        # 'homeless_shelter_reach': 'homeless_shelter_count',
        # 'traffic_signal_reach': 'traffic_signal_count',
        # 'street_lighting_poles_reach': 'street_lighting_poles_count',
        # 'schools_reach': 'schools_count'
    }, 
    lambda dst: normal_dst(dst, STANDARD_DEVIATION),
    limit=1000
)
junctions.write_to_file('future_dataset_reach.csv')

Calculated 6179/6179           
Normalizing
Done


**Merging Future Dataset with new dataset that has more features**

In [4]:
# Columns to extract: 'neighborhood_id', 'Census Population', 'English (as mother tongue)', 'Unemployment Rate', 'Public Transport', 'Median Household Income', 'Population in Low-Income Households'

import pandas as pd

# Load datasets
merf_df = pd.read_excel('data\\merged_final_with_updated_features.xlsx')
future_df = pd.read_excel('data\\future_dataset.xlsx')

# Filter for the year 2023
merf_2023_df = merf_df[merf_df['Year'] == 2023]

# Select the required columns
columns_to_extract = ['id', 'neighborhood_id', 'Census Population', 'English (as mother tongue)', 
                      'Unemployment Rate', 'Public Transport', 'Median Household Income', 
                      'Population in Low-Income Households']
merf_2023_selected_df = merf_2023_df[columns_to_extract]

# Merge datasets on the 'id' column
merged_df = pd.merge(future_df, merf_2023_selected_df, on='id', how='left')

# Save the merged dataframe to a new Excel file
merged_df.to_excel('data\\future_dataset_merged.xlsx', index=False)


**Double checking the transit reaches are changing in a way that makes sense**

In [2]:
import pandas as pd

# Load datasets
merged_final_file = "data\\merged_final_with_updated_features.xlsx"
future_dataset_file = "data\\future_dataset_merged.xlsx"

merged_final_df = pd.read_excel(merged_final_file)
future_dataset_df = pd.read_excel(future_dataset_file)

# Filter merged_final_df for Year == 2023
merged_final_2023 = merged_final_df[merged_final_df['Year'] == 2023]

# Calculate differences for each junction
for index, row in merged_final_2023.iterrows():
    junction = row['id']
    transit_reach_2023 = row['transit_reach']
    
    # Find corresponding row in future_dataset_df
    future_row = future_dataset_df[future_dataset_df['id'] == junction]
    
    if not future_row.empty:
        future_dataset_df.loc[future_dataset_df['id'] == junction, 'transit_reach_difference'] = future_row['transit_reach'].values[0] - transit_reach_2023

# Save the updated DataFrame if needed
output_file = "future_dataset_merged_with_difference.xlsx"
future_dataset_df.to_excel(output_file, index=False)

# Display the updated DataFrame
print(future_dataset_df.head())


   id      type  year  street_count   longitude   latitude  elevation  \
0   1  Junction  2024             2 -123.224758  49.274760         65   
1   4  Junction  2024             4 -123.220892  49.271168         90   
2   5  Junction  2024             3 -123.220567  49.273832         71   
3   6  Junction  2024             3 -123.219613  49.268957         93   
4   7  Junction  2024             2 -123.219572  49.267203         97   

   average_segments_length  betweenness_centrality  \
0               396.227813            0.000000e+00   
1               480.393636            2.100000e-07   
2               522.393980            3.214230e-04   
3               265.464150            3.203230e-04   
4               261.840633            4.610000e-06   

   scaled_betweenness_centrality  ... street_lighting_poles_reach  \
0                       0.000000  ...                    0.024386   
1                       0.020964  ...                    0.067050   
2                      32.142